In [1]:
from pyspark.sql import SparkSession
import os
import configparser
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1608056173530_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create spark session with hadoop-aws package
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# set input and output file paths
input_data = "s3a://udacity-dend/"
output_data = "s3a://udacity-data-lake-w-spark/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read song data files
song_data = input_data + 'song_data/*/*/*/*.json'
songs_df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
songs_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [6]:
songs_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [7]:
songs_df.createOrReplaceTempView("songs_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# extract columns to create songs table
songs_table = spark.sql('''
    select distinct song_id, title, artist_id, year, duration
    from songs_data where song_id is not null
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
songs_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [10]:
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOSJVZR12AB018698F|Four Simple Steps...|ARJ8IIY1187B992734|2003|  58.8273|
|SONXXYP12A81C238EE|Don't Waste My Ti...|ARZNYC61187B9958CF|1985|324.85832|
|SOTSGLP12AB017F7D7|The Sexy Midnight...|ARNCVXP1187FB54C24|2009| 268.9824|
|SOOKGUZ12AC960DEA2|(The Man With The...|AR7X9J51187FB3F1A9|2004|200.30649|
|SOGREWP12A58A78AB7|I've Got the Worl...|ARQ6PLP1187FB468E3|2006|140.90404|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

In [37]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy("year", "artist_id").mode('overwrite').parquet(output_data + "songs.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# extract columns to create artists table
artists_table = spark.sql('''
    select distinct artist_id, artist_name, artist_location, artist_latitude, artist_longitude
    from songs_data where artist_id is not null
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
artists_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10025

In [14]:
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+--------------------+---------------+----------------+
|         artist_id|      artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+-----------------+--------------------+---------------+----------------+
|ART1XOT1187B99A298|     Billy Squier| Wellesley Hills, MA|        42.3076|       -71.27951|
|ARB57BN1187B9B5EAF|    36 Crazyfists|       Anchorage, AK|       61.21756|      -149.85776|
|AROOKC21187B9AE85C|    Missy Higgins|Melbourne, Australia|      -37.81753|       144.96715|
|AR1ZYLH1187B98C159|Naughty By Nature|     East Orange, NJ|           null|            null|
|ARUYIJ01187FB51CF6|     The Kingsmen|        Portland, OR|       45.51179|      -122.67563|
+------------------+-----------------+--------------------+---------------+----------------+
only showing top 5 rows

In [15]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data +  "artists.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# read log data files
log_data = input_data + 'log_data/*/*/*.json'
df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df.createOrReplaceTempView("log_data")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [19]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [20]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [21]:
# extract columns to create sousers table
users_table = spark.sql('''
    SELECT distinct cast(userid as INT) as user_id, firstname as first_name, lastname as last_name, gender, level 
    from log_data where page = 'NextSong' and userid is not null
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
users_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

In [23]:
users_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

104

In [24]:
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     26|      Ryan|    Smith|     M| free|
|     49|     Chloe|   Cuevas|     F| free|
|     29|Jacqueline|    Lynch|     F| free|
|     15|      Lily|     Koch|     F| paid|
|      8|    Kaylee|  Summers|     F| free|
+-------+----------+---------+------+-----+
only showing top 5 rows

In [25]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data +  "users.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# extract columns to create time table
time_table = spark.sql('''
     SELECT distinct to_timestamp(ts/1000) as start_time
     ,extract(hour from (to_timestamp(ts/1000))) as hour
     ,extract(day from (to_timestamp(ts/1000))) as day
     ,extract(week from (to_timestamp(ts/1000))) as week
     ,extract(month from (to_timestamp(ts/1000))) as month
     ,extract(year from (to_timestamp(ts/1000))) as year
     ,date_format(to_timestamp(ts/1000),'EEEE') as weekday                        
     from log_data where page = 'NextSong' and ts is not null
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: string (nullable = true)

In [28]:
time_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6813

In [29]:
time_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-05 09:54:...|   9|  5|  45|   11|2018| Monday|
|2018-11-05 10:41:...|  10|  5|  45|   11|2018| Monday|
|2018-11-30 16:25:...|  16| 30|  48|   11|2018| Friday|
|2018-11-30 18:03:...|  18| 30|  48|   11|2018| Friday|
|2018-11-16 08:38:...|   8| 16|  46|   11|2018| Friday|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [30]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").mode('overwrite').parquet(output_data + "time.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# join log and song datasets and extract columns for songplays table
songplays_table = spark.sql('''
    SELECT  
	to_timestamp(l.ts/1000) as start_time,
    extract(year from (to_timestamp(l.ts/1000))) as start_year,
	extract(month from (to_timestamp(ts/1000))) as start_month,
	cast(l.userid as INT) as user_id, 
	l.level, 
	s.song_id, 
	s.artist_id, 
	l.sessionid, 
	l.location, 
	l.useragent
    from log_data l 
    left join songs_data s on l.song=s.title and l.artist=s.artist_name
    where l.page = 'NextSong'
    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# add auto-increment ID column to songplays table
songplays_table = songplays_table.withColumn("songplay_id", F.monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- start_month: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- sessionid: long (nullable = true)
 |-- location: string (nullable = true)
 |-- useragent: string (nullable = true)
 |-- songplay_id: long (nullable = false)

In [34]:
songplays_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

In [35]:
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-----------+-------+-----+-------+---------+---------+--------------------+--------------------+-----------+
|          start_time|start_year|start_month|user_id|level|song_id|artist_id|sessionid|            location|           useragent|songplay_id|
+--------------------+----------+-----------+-------+-----+-------+---------+---------+--------------------+--------------------+-----------+
|2018-11-15 00:30:...|      2018|         11|     26| free|   null|     null|      583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|          0|
|2018-11-15 00:41:...|      2018|         11|     26| free|   null|     null|      583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|          1|
|2018-11-15 00:45:...|      2018|         11|     26| free|   null|     null|      583|San Jose-Sunnyval...|"Mozilla/5.0 (X11...|          2|
|2018-11-15 03:44:...|      2018|         11|     61| free|   null|     null|      597|Houston-The Woodl...|"Mozilla/5.0 (Mac...|          3|
|2018-

In [36]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy("start_year", "start_month").mode('overwrite').parquet(output_data + "songplays.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…